In [1]:
import http.client
import pandas as pd
import requests
import json
from multiprocess.pool import ThreadPool

In [2]:
data_movies = pd.read_excel("IMDB_URL___6137.xlsx")

In [3]:
data_movies.head()

,Name,URL,ID Long,ID,ID.1
0,Parasite,https://www.imdb.com/title/tt6751668/?ref_=adv...,https://www.imdb.com/title/tt6751668,tt6751668,tt6751668
1,Oldboy,https://www.imdb.com/title/tt0364569/?ref_=adv...,https://www.imdb.com/title/tt0364569,tt0364569,tt0364569
2,Train to Busan,https://www.imdb.com/title/tt5700672/?ref_=adv...,https://www.imdb.com/title/tt5700672,tt5700672,tt5700672
3,The Handmaiden,https://www.imdb.com/title/tt4016934/?ref_=adv...,https://www.imdb.com/title/tt4016934,tt4016934,tt4016934
4,Minari,https://www.imdb.com/title/tt10633456/?ref_=ad...,https://www.imdb.com/title/tt1063345,tt1063345,tt1063345


In [4]:
data_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6137 entries, 0 to 6136
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Name     6137 non-null   object
 1   URL      6137 non-null   object
 2   ID Long  6137 non-null   object
 3   ID       6137 non-null   object
 4   ID.1     6137 non-null   object
dtypes: object(5)
memory usage: 239.9+ KB


In [5]:
movie_id_list = data_movies["ID"].tolist()

In [6]:
movie_id_list_test = movie_id_list[700:800]

In [7]:
conn = http.client.HTTPSConnection("imdb-api.com", 443)
payload = ''
headers = {}

In [8]:
list_of_results = []

In [9]:
#columns = ["Movie", "Poster_url", "Trailer_url"]   
columns = ["ID", "Title", "Original Title", "Year", "Runtime_mins", "Director", "Plot", "Poster_url", "Trailer_url", "YouTube_url", "Budget", "OpeningWeekendUSA",
          "GrossUSA", "World"] 

In [10]:
len(columns)

14

In [11]:
# set of budget types
set_of_budget = set()

In [12]:
 def func(movie_id):   
    

    #print(movie_id)
    http_posters = "https://imdb-api.com/en/API/Posters/k_f4uhd34i/" + movie_id
    http_trailer = "https://imdb-api.com/en/API/Trailer/k_f4uhd34i/" + movie_id
    http_titel = "https://imdb-api.com/en/API/Title/k_f4uhd34i/" + movie_id + "/FullActor,FullCast,Posters,Images,Trailer,Ratings,Wikipedia,"
    http_general = "https://imdb-api.com/en/API/Title/k_f4uhd34i/" + movie_id
    http_youtube = "https://imdb-api.com/en/API/YouTubeTrailer/k_f4uhd34i/" + movie_id

    payload = {}
    headers= {}

    response_posters = requests.request("GET", http_posters, headers=headers, data = payload)
    response_trailer = requests.request("GET", http_trailer, headers=headers, data = payload)
    response_general = requests.request("GET", http_general, headers=headers, data=payload)
    response_youtube = requests.request("GET", http_youtube, headers=headers, data=payload)

    response_posters.raise_for_status()
    response_trailer.raise_for_status()
    response_general.raise_for_status()
    response_youtube.raise_for_status()
    
    
    if (response_posters.status_code != 204)  & (response_trailer.status_code != 204) &\
    (response_general.status_code != 204) & (response_youtube.status_code != 204):

        json_posters = response_posters.json()
        json_trailer = response_trailer.json()
        json_general = response_general.json()
        json_youtube = response_youtube.json()


    try:

        link_first_poster = json_posters['posters'][0]['link']
    except:
        link_first_poster = "N/A"

    try:
        link_trailer = json_trailer['link']
    except:
        link_trailer = "N/A"

    try:    
        link_youtube = json_youtube['videoUrl']
    except:
        link_youtube = "N/A"

    try:
        title = json_trailer['title']
    except:
        title = "N/A"

    try:
        movie_id = json_general['id']
    except:
        movie_id = "N/A"

    try:
        year = json_general['year']
    except:
        year = "N/A"

    try:
        original_title = json_general['originalTitle']
    except:
        original_title = "N/A"

    try:
        runtime = json_general['runtimeMins']
    except:
        runtime = "N/A"

    try:
        directors = json_general['directors']
    except:
        directors = "N/A"

    try:
        plot = json_general['plot']
    except:
        plot = "N/A"

    try:
        budget = json_general['boxOffice']['budget']
    except:
        budget = "N/A"

    try:
        open_usa = json_general['boxOffice']['openingWeekendUSA']
    except:
        open_usa = "N/A"

    try:
        gross_usa = json_general['boxOffice']['grossUSA']
    except:
        gross_usa = "N/A"

    try:
        world = json_general['boxOffice']['cumulativeWorldwideGross']
    except:
        world = "N/A"

    # add budget types to a set
    try:
        for i in json_general['boxOffice'].keys():
            set_of_budget.add(i)
    except:
        pass

    row = pd.Series([movie_id, title, original_title, year, runtime, directors, plot, link_first_poster, link_trailer, link_youtube, budget, open_usa, gross_usa, world],index = columns) 
    return row

In [13]:
def main():
    pool = ThreadPool(processes=8)
    parse_list = pool.map(func, movie_id_list_test)
    df = pd.DataFrame(parse_list)
    return df
    
if __name__ == '__main__':
    df_parsed = main()

In [14]:
df_parsed

,ID,Title,Original Title,Year,Runtime_mins,Director,Plot,Poster_url,Trailer_url,YouTube_url,Budget,OpeningWeekendUSA,GrossUSA,World
0,tt0364447,My Wife Is a Gangster 2,,2003,110,Heung-sun Jeong,Former Scissor Gang leader Eun-jin (Shin Eun-k...,https://imdb-api.com/posters/original/abFUjJZH...,https://www.imdb.com/video/vi1135062553,,,,,
1,tt6985476,Mothers,Dangshinui Bootak,2017,119,Dong-Eun Lee,Hyo Jin is a 32-year-old woman whose husband d...,https://imdb-api.com/posters/original/xMeE7SkM...,None,https://www.youtube.com/watch?v=d4wi5tPvRrk,,,,
2,tt1293743,In Berlin,,2009,97,"Michael Ballhaus, Ciro Cappellari",By accompanying 20 very different individuals ...,https://imdb-api.com/posters/original/gFKwMduc...,None,,,,,
3,tt6214734,One Day,Eoneunal,2017,118,Yoon-ki Lee,Insurance examiner Gang-Soo (Kim Nam-Gil) fall...,https://imdb-api.com/posters/original/anrjtPUU...,None,https://www.youtube.com/watch?v=EYTM1EuZvso,,,,"$1,534,329"
4,tt7299346,Love+Sling,Leo-beu-seul-ling,2018,110,Dae-Woong Kim,This is the story of a father and son who grow...,https://imdb-api.com/posters/original/jDBKygZb...,None,https://www.youtube.com/watch?v=sYx9g1629Cs,,,,"$5,823,505"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,tt3694790,Steal My Heart,,2013,115,Hyun-Jong Lee,Hot-shot police detective Lee is out to get a ...,https://imdb-api.com/posters/original/5dRo8P7E...,None,https://www.youtube.com/watch?v=dTYc-a1p1D8,,,,"$3,032,434"
96,tt0419135,Shinbu sueob,,2004,110,In-mu Heo,"Kyu-sik is a seminary student, who one day fal...",https://imdb-api.com/posters/original/8IPYhz6Y...,None,,,,,"$251,561"
97,tt0475557,Yu-wol-ui il-gi,,2005,105,Kyung-Soo Im,Two teenagers joining the same class in a high...,https://imdb-api.com/posters/original/tMqmbVAS...,None,,,,,"$3,867,958"
98,tt0108192,Sopyonje,Seopyeonje,1993,112,Kwon-taek Im,The specifically Korean tradition that is recl...,https://imdb-api.com/posters/original/tCejyq7o...,None,https://www.youtube.com/watch?v=fu6gmh4Wvys,,,,
